In [45]:
from __future__ import print_function
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import json
import tensorflow_datasets as tfds

In [60]:
#讀檔

num = 0
train_data = []
test_data = []
train_size = 10000
test_size = 100
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.

input_file = open('translation2019zh_train.json', encoding = 'utf8')
for line in input_file.readlines():
    num = num + 1
    if num > train_size + test_size:
        break
    elif num > train_size:
        test_data.append(json.loads(line))
    else:
        train_data.append(json.loads(line))
input_file.close()

#test_data[:10]
#dataset = tf.data.Dataset.from_tensor_slices((test_data[line]['english'],test_data[line]['chinese']))

In [62]:
dataset_e=[]
#type(test_data[2]['english'])
type(dataset_e)
#dataset_e[1]['english']
#dataset = tf.data.Dataset.from_tensor_slices((dataset_e,dataset_c))


list

In [ ]:
#读取cmn-eng.txt文件
data_path = 'data/cmn.txt'
df = pd.read_table(data_path,header=None).iloc[:NUM_SAMPLES,:,]
df.columns=['inputs','targets']
#讲每句中文句首加上'\t'作为起始标志，句末加上'\n'作为终止标志
df['targets'] = df['targets'].apply(lambda x: '\t'+x+'\n')

input_texts = df.inputs.values.tolist()#英文句子列表
target_texts = df.targets.values.tolist()#中文句子列表

#确定中英文各自包含的字符。df.unique()直接取sum可将unique数组中的各个句子拼接成一个长句子
input_characters = sorted(list(set(df.inputs.unique().sum())))
target_characters = sorted(list(set(df.targets.unique().sum())))

In [7]:
# 讀取資料檔，並將所有單字整理為字典，分別為英文及中文字典，注意，英文為字母的集合，非單字(Word)
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for item in train_data:
    input_text = item['english']
    target_text = '\t' + item['chinese'] + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [42]:
#type(input_text)
dataset = tf.data.Dataset.from_tensor_slices((input_texts,target_texts))


In [49]:
type(dataset)

tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

In [55]:
builder = tfds.builder("wmt_translate_ende")
type(builder)

tensorflow_datasets.translate.wmt_ende.WmtTranslateEnde

In [56]:
# 讀取資料檔，並將所有單字整理為字典，分別為英文及中文字典，注意，英文為字母的集合，非單字(Word)
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for item in train_data:
    input_text = item['english']
    target_text = '\t' + item['chinese'] + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [59]:
print(input_texts[:10])
type(input_texts)

['For greater sharpness, but with a slight increase in graininess, you can use a 1:1 dilution of this developer.', 'He calls the Green Book, his book of teachings, “the new gospel.', 'And the light breeze moves me to caress her long ear', 'They have the blood of martyrs is the White to flow …', "Finally, the Lakers head to the Motor City to take on a Pistons team that currently owns the Eastern Conference's second best record (1/31). L.", '"The perfect match—my father loves names and Jackie loves money, " sneered Alexander at the wedding. Neither he nor Christina ever got along with their stepmother17.', 'In 2006, Walmart was charged with racism when its recommendation engine paired Planet of the Apes with a documentary about Martin Luther King.', 'The matte as main copper phase in the cleaning. slag was deter- mined by electron probe microscopic analysis.', 'Have you shined your shoes?', 'The Tanning Matrix can be formed by resorcinol and oxazolidine E, and the reactioncharateristics 

list

In [7]:
# 字典排序            
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

In [8]:
# 計算編碼器、解碼器的最大長度
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [9]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 439
Number of unique output tokens: 4031
Max sequence length for inputs: 295
Max sequence length for outputs: 182


In [10]:
# 以dict儲存字典單字及序號
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [11]:
# 設定編碼器、解碼器input起始值(均為0矩陣)
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [12]:
print(input_token_index)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, '&': 6, "'": 7, '(': 8, ')': 9, '*': 10, '+': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23, '8': 24, '9': 25, ':': 26, ';': 27, '<': 28, '=': 29, '>': 30, '?': 31, 'A': 32, 'B': 33, 'C': 34, 'D': 35, 'E': 36, 'F': 37, 'G': 38, 'H': 39, 'I': 40, 'J': 41, 'K': 42, 'L': 43, 'M': 44, 'N': 45, 'O': 46, 'P': 47, 'Q': 48, 'R': 49, 'S': 50, 'T': 51, 'U': 52, 'V': 53, 'W': 54, 'X': 55, 'Y': 56, 'Z': 57, '[': 58, '\\': 59, ']': 60, '^': 61, '_': 62, '`': 63, 'a': 64, 'b': 65, 'c': 66, 'd': 67, 'e': 68, 'f': 69, 'g': 70, 'h': 71, 'i': 72, 'j': 73, 'k': 74, 'l': 75, 'm': 76, 'n': 77, 'o': 78, 'p': 79, 'q': 80, 'r': 81, 's': 82, 't': 83, 'u': 84, 'v': 85, 'w': 86, 'x': 87, 'y': 88, 'z': 89, '~': 90, '«': 91, '\xad': 92, '´': 93, '·': 94, 'Â': 95, 'à': 96, 'á': 97, 'â': 98, 'ä': 99, 'è': 100, 'é': 101, 'ñ': 102, 'ó': 103, 'ö': 104, 'ü': 105, 'δ': 106, 'τ': 107, '–': 108, '—': 109, '‘': 

In [13]:
print(target_token_index)

{'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, '#': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, '*': 12, '+': 13, ',': 14, '-': 15, '.': 16, '/': 17, '0': 18, '1': 19, '2': 20, '3': 21, '4': 22, '5': 23, '6': 24, '7': 25, '8': 26, '9': 27, ':': 28, ';': 29, '<': 30, '=': 31, '>': 32, '?': 33, 'A': 34, 'B': 35, 'C': 36, 'D': 37, 'E': 38, 'F': 39, 'G': 40, 'H': 41, 'I': 42, 'J': 43, 'K': 44, 'L': 45, 'M': 46, 'N': 47, 'O': 48, 'P': 49, 'Q': 50, 'R': 51, 'S': 52, 'T': 53, 'U': 54, 'V': 55, 'W': 56, 'X': 57, 'Y': 58, 'Z': 59, '[': 60, '\\': 61, ']': 62, '^': 63, '_': 64, 'a': 65, 'b': 66, 'c': 67, 'd': 68, 'e': 69, 'f': 70, 'g': 71, 'h': 72, 'i': 73, 'j': 74, 'k': 75, 'l': 76, 'm': 77, 'n': 78, 'o': 79, 'p': 80, 'q': 81, 'r': 82, 's': 83, 't': 84, 'u': 85, 'v': 86, 'w': 87, 'x': 88, 'y': 89, 'z': 90, '~': 91, '\xad': 92, '·': 93, '»': 94, 'Ñ': 95, 'á': 96, 'ä': 97, 'ñ': 98, 'ö': 99, 'ü': 100, '–': 101, '—': 102, '―': 103, '‘': 104, '’': 105, '“': 106, '”': 107, '•': 108, '…': 109, '‧'

In [14]:
# 設定 encoder_input、decoder_input對應的順序    
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [15]:
# 建立 encoder LSTM 隱藏層
origin_encoder_inputs = Input(shape=(None, num_encoder_tokens), name='input_0')
origin_encoder = LSTM(latent_dim, return_state=True)
origin_encoder_outputs, origin_state_h, origin_state_c = origin_encoder(origin_encoder_inputs)
origin_encoder_states = [origin_state_h, origin_state_c]

encoder_inputs = Input(shape=(None, num_encoder_tokens), name='input_1')
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
print(encoder_outputs)
print(state_h)
print(state_c)
encoder_outputs.type_spec

KerasTensor(type_spec=TensorSpec(shape=(None, 256), dtype=tf.float32, name=None), name='lstm_1/PartitionedCall:0', description="created by layer 'lstm_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 256), dtype=tf.float32, name=None), name='lstm_1/PartitionedCall:2', description="created by layer 'lstm_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 256), dtype=tf.float32, name=None), name='lstm_1/PartitionedCall:3', description="created by layer 'lstm_1'")


TensorSpec(shape=(None, 256), dtype=tf.float32, name=None)

In [16]:
# 捨棄 output，只保留記憶狀態 h 及 c
encoder_states = [state_h, state_c]

In [17]:
# 建立 decoder LSTM 隱藏層
decoder_inputs = Input(shape=(None, num_decoder_tokens), name='input_2')
# We set up our decoder to return full output sequences,
# decoder 記憶狀態不會在訓練過程使用，只會在推論(Inference)使用
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [18]:
# 定義模型，由 encoder_input_data 及 decoder_input_data 轉換為 decoder_target_data 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [19]:
# 訓練
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/10
125/125 [==============================] - 395s 3s/step - loss: 1.3938 - val_loss: 1.3592
Epoch 2/10
125/125 [==============================] - 373s 3s/step - loss: 1.3697 - val_loss: 1.3544
Epoch 3/10
125/125 [==============================] - 384s 3s/step - loss: 1.3652 - val_loss: 1.3491
Epoch 4/10
125/125 [==============================] - 391s 3s/step - loss: 1.3633 - val_loss: 1.3494
Epoch 5/10
125/125 [==============================] - 398s 3s/step - loss: 1.3623 - val_loss: 1.3504
Epoch 6/10
125/125 [==============================] - 400s 3s/step - loss: 1.3619 - val_loss: 1.3488
Epoch 7/10
125/125 [==============================] - 400s 3s/step - loss: 1.3613 - val_loss: 1.3488
Epoch 8/10
125/125 [==============================] - 404s 3s/step - loss: 1.3608 - val_loss: 1.3483
Epoch 9/10
125/125 [==============================] - 395s 3s/step - loss: 1.3602 - val_loss: 1.3463
Epoch 10/10
125/125 [==============================] - 386s 3s/step - loss: 1.3593 - val_lo

In [20]:
# 定義編碼器取樣模型
encoder_model = Model(encoder_inputs, encoder_states)

In [21]:
# 定義解碼器的input
decoder_state_input_h = Input(shape=(latent_dim,), name='input_3')
decoder_state_input_c = Input(shape=(latent_dim,), name='input_4')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [22]:
# 定義解碼器 LSTM 模型
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

In [23]:
# 以編碼器的記憶狀態 h 及 c 為解碼器的記憶狀態  
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

print([decoder_inputs] + decoder_states_inputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
'''
decoder_model = Model(inputs=[decoder_inputs].append(decoder_states_inputs), outputs=[decoder_outputs].append(decoder_states))
'''

[<KerasTensor: shape=(None, None, 4031) dtype=float32 (created by layer 'input_2')>, <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'input_3')>, <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'input_4')>]


'\ndecoder_model = Model(inputs=[decoder_inputs].append(decoder_states_inputs), outputs=[decoder_outputs].append(decoder_states))\n'

In [24]:
# 建立反向的 dict，才能透過查詢將數值轉回文字
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [25]:
# 模型預測，並取得翻譯結果(中文)    
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [26]:
test_texts = []
test_characters = set()
for text in test_data:
    test_texts.append(text['english'])
    for char in text['english']:
        if char not in test_characters:
            test_characters.add(char)

print(test_texts)
print(test_characters)
print(input_characters)

['Parent Richemont initiated a hiring freeze and job cuts to help it overcome the crisis, which Fornas said was mostly in the past. "The worst, I think, is hopefully behind us."', 'The groups said that they will talk to the new democratic government in the future about transforming themselves into border guard forces.', 'We had a bunker in The Guardian offices where we — The Guardian, [the] New York Times and Der Spiegel reporters [were].', 'The koala was a dead weight holding me down and we stayed in those brown dark depths for what seemed like half an eternity.', 'This beautiful two story home is located in the Rowland Unified School District with four National Blue Ribbon Schools.', 'The bulk of our nutritional needs come from fruits, vegetables, and grains.', '"Whoever attacks us, " the prime minister said, "his blood will be on his own head. "', 'In multiple randomized trials, including THA and TKA patients, LMWH has been more effective than warfarin in limiting overall DVT rates.

In [27]:
test_characters = sorted(list(test_characters))
num_test = len(test_characters)

In [28]:
test_index = dict([(char, i) for i, char in enumerate(test_characters)])

In [29]:
test_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

In [30]:
# 設定 encoder_input、decoder_input對應的順序    
for i, test_text in enumerate(test_texts):
    for t, char in enumerate(test_text):
        #print(char)
        #print(input_token_index[char])
        test_input_data[i, t, input_token_index[char]] = 1.

In [31]:
print(len(test_input_data))

10000


In [32]:
print(test_index)

{' ': 0, '"': 1, '$': 2, "'": 3, '(': 4, ')': 5, ',': 6, '-': 7, '.': 8, '0': 9, '1': 10, '2': 11, '3': 12, '4': 13, '5': 14, '6': 15, '7': 16, '8': 17, '9': 18, ':': 19, ';': 20, '?': 21, 'A': 22, 'B': 23, 'C': 24, 'D': 25, 'E': 26, 'F': 27, 'G': 28, 'H': 29, 'I': 30, 'J': 31, 'K': 32, 'L': 33, 'M': 34, 'N': 35, 'O': 36, 'P': 37, 'Q': 38, 'R': 39, 'S': 40, 'T': 41, 'U': 42, 'V': 43, 'W': 44, 'X': 45, 'Y': 46, 'Z': 47, '[': 48, ']': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, '—': 76, '‘': 77, '’': 78, '。': 79, '￡': 80}


In [33]:
print(input_token_index)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, '&': 6, "'": 7, '(': 8, ')': 9, '*': 10, '+': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23, '8': 24, '9': 25, ':': 26, ';': 27, '<': 28, '=': 29, '>': 30, '?': 31, 'A': 32, 'B': 33, 'C': 34, 'D': 35, 'E': 36, 'F': 37, 'G': 38, 'H': 39, 'I': 40, 'J': 41, 'K': 42, 'L': 43, 'M': 44, 'N': 45, 'O': 46, 'P': 47, 'Q': 48, 'R': 49, 'S': 50, 'T': 51, 'U': 52, 'V': 53, 'W': 54, 'X': 55, 'Y': 56, 'Z': 57, '[': 58, '\\': 59, ']': 60, '^': 61, '_': 62, '`': 63, 'a': 64, 'b': 65, 'c': 66, 'd': 67, 'e': 68, 'f': 69, 'g': 70, 'h': 71, 'i': 72, 'j': 73, 'k': 74, 'l': 75, 'm': 76, 'n': 77, 'o': 78, 'p': 79, 'q': 80, 'r': 81, 's': 82, 't': 83, 'u': 84, 'v': 85, 'w': 86, 'x': 87, 'y': 88, 'z': 89, '~': 90, '«': 91, '\xad': 92, '´': 93, '·': 94, 'Â': 95, 'à': 96, 'á': 97, 'â': 98, 'ä': 99, 'è': 100, 'é': 101, 'ñ': 102, 'ó': 103, 'ö': 104, 'ü': 105, 'δ': 106, 'τ': 107, '–': 108, '—': 109, '‘': 

In [34]:
# 測試100次
for seq_index in range(len(test_texts)):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = test_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(seq_index + 1, '.')
    print('English:', test_texts[seq_index])
    try:
        print('Chinese:', decoded_sentence)
    except:
        # 出現亂碼，以?取代
        print('Chinese:', decoded_sentence.encode('ascii', 'replace'))
        #print("error:", sys.exc_info()[0])


1 .
English: Parent Richemont initiated a hiring freeze and job cuts to help it overcome the crisis, which Fornas said was mostly in the past. "The worst, I think, is hopefully behind us."
Chinese: 这，，，，的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
2 .
English: The groups said that they will talk to the new democratic government in the future about transforming themselves into border guard forces.
Chinese: 这，，，，的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
3 .
English: We had a bunker in The Guardian offices where we — The Guardian, [the] New York Times and Der Spiegel reporters [were].
Chinese: 这，，，，的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的

26 .
English: On the market a lot of the name of the decoration decoration company, is often not related to the quality of the.
Chinese: 这，，，，的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
27 .
English: Here is your steak, enjoy it please.
Chinese: 这，，，，的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
28 .
English: Rather, it is a way to use potentially explosive energy to draw a response from the uke.
Chinese: 这，，，，的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
29 .
English: Learn how to flip a paper boat inside out with tips from an origami enthusiast in this free origami video.
Chinese: 这，，，，的的的的

53 .
English: We need emotions to make decisions - as in all things, balance is the key.
Chinese: 这，，，，的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
54 .
English: Alpha thalassaemia, an inherited condition, is common in many areas of the world where malaria is endemic, particularly Africa and southeast Asia.
Chinese: 这，，，，的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
55 .
English: He was completely honest in his reply.
Chinese: 这，，，，的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
56 .
English: It was this journey that changed Jiang Hao’s life radically.
Chinese: 这，，，，的的的的的的的的的的的的的的的的的的的的的的的的的的的的的

79 .
English: Conclusion Pseudomonas aeruginosa vaccine could ease local infection in injection site and constitutional symptom, and it could block the disseminated infections by Candida albicans in kidney.
Chinese: 这，，，，的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
80 .
English: Effect:This spell conjures a barrier of divine energy to protect you. While encased in this barrier you cannot activate other abilities and cannot be affected by any spells.
Chinese: 这，，，，的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
81 .
English: Base 8 languages are rare in the world, but it does appear that early Dravidian is base 8, but later changed to base 10 (possibly under Indo-European influence).
Chinese: 这，，，，的的的的的的的的的的的的的的的的的的的的的的的的的的的的的

In [35]:
print(test_data)

[{'english': 'Parent Richemont initiated a hiring freeze and job cuts to help it overcome the crisis, which Fornas said was mostly in the past. "The worst, I think, is hopefully behind us."', 'chinese': '此前卡地亚母公司Richemont通过冻结招聘和裁员来度过此次危机,对此Fornas表示这已成为过去,"我相信最糟糕的时刻已经过去".(完)'}, {'english': 'The groups said that they will talk to the new democratic government in the future about transforming themselves into border guard forces.', 'chinese': '他们表示将与新的民主政府讨论关于改编成为边境警卫部队。'}, {'english': 'We had a bunker in The Guardian offices where we — The Guardian, [the] New York Times and Der Spiegel reporters [were].', 'chinese': '阿桑杰：我们在《卫报》的办公室中也设置了一个类似的“掩体”，把《卫报》、《纽约时报》和《明镜周刊》三家媒体的记者都聚到那里。'}, {'english': 'The koala was a dead weight holding me down and we stayed in those brown dark depths for what seemed like half an eternity.', 'chinese': '它死沉死沉的，把我一直往下坠。 我和它呆在黑魆魆的水下，仿佛看见地狱之门半开着。'}, {'english': 'This beautiful two story home is located in the Rowland Unified School District with four National Blue 

# 不知道為何跑不出來